In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-and-burnout-in-the-workplace/mental_health_workplace_survey.csv


In [87]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [88]:
#input file
file_path = "/kaggle/input/mental-health-and-burnout-in-the-workplace/mental_health_workplace_survey.csv"
df = pd.read_csv(file_path)

In [89]:
#cleaned file
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip().str.lower()


df.replace(['', 'na', 'n/a', 'none', 'unknown'], np.nan, inplace=True)

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)


cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)


In [90]:
#DAY4
#Cleaning and encoding
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].fillna("Unknown")
    df[col] = LabelEncoder().fit_transform(df[col])


In [91]:
#Feature and target
X = df.drop(columns=["BurnoutRisk", "EmployeeID"])
y = df["BurnoutRisk"]


In [92]:
#splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=69)

#scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [93]:
#trainng Decision Tree, Random Forest, k-Nearest Neighbors models

dt = DecisionTreeClassifier(random_state=69)
rf = RandomForestClassifier(random_state=69)
knn = KNeighborsClassifier()

dt.fit(X_train_scaled, y_train)
rf.fit(X_train_scaled, y_train)
knn.fit(X_train_scaled, y_train)

#accuracy using all features
acc_dt = accuracy_score(y_test, dt.predict(X_test_scaled))
acc_rf = accuracy_score(y_test, rf.predict(X_test_scaled))
acc_knn = accuracy_score(y_test, knn.predict(X_test_scaled))

print("Accuracy")
print("Decision Tree Accuracy:", acc_dt)
print("Random Forest Accuracy:", acc_rf)
print("k-NN Accuracy:", acc_knn)

Accuracy
Decision Tree Accuracy: 1.0
Random Forest Accuracy: 1.0
k-NN Accuracy: 0.8333333333333334


In [94]:
#selecting the best 3 features 
top_3 = rf_df.head(3)['Feature'].tolist()
print("Top 3 Features:", top_3)

X_train_3 = X_train[top_3]
X_test_3 = X_test[top_3]

X_train_3_scaled = scaler.fit_transform(X_train_3)
X_test_3_scaled = scaler.transform(X_test_3)

Top 3 Features: ['BurnoutLevel', 'WorkLifeBalanceScore', 'CareerGrowthScore']


In [95]:
#retraining the models with the best 3 features to compare
dt.fit(X_train_3_scaled, y_train)
rf.fit(X_train_3_scaled, y_train)
knn.fit(X_train_3_scaled, y_train)

# Accuracy using top 3 features
acc_best_dt = accuracy_score(y_test, dt.predict(X_test_3_scaled))
acc_best_rf = accuracy_score(y_test, rf.predict(X_test_3_scaled))
acc_best_knn = accuracy_score(y_test, knn.predict(X_test_3_scaled))


print("Accuracy with Top 3 Features:")
print("Decision Tree:", acc_best_dt)
print("Random Forest:", acc_best_rf)
print("k-NN:", acc_best_knn)


Accuracy with Top 3 Features:
Decision Tree: 1.0
Random Forest: 1.0
k-NN: 0.9786666666666667


In [96]:
#Comparison b/w the one with all feature and top 3 features 
result_df = pd.DataFrame({
    "Model": ["Decision Tree", "Random Forest", "k-NN"],
    "Accuracy (All Features)": [acc_dt, acc_rf, acc_knn],
    "Accuracy (Top 3 Features)": [acc_best_dt, acc_best_rf, acc_best_knn]
})

result_df


,Model,Accuracy (All Features),Accuracy (Top 3 Features)
0,Decision Tree,1.000000,1.000000
1,Random Forest,1.000000,1.000000
2,k-NN,0.833333,0.978667


SUMMARY

Decision Tree and Random Forest both achieved 100% accuracy using all features and also using just the top 3 features — meaning feature reduction did not affect their performance.

k-NN improved from 83.3% accuracy (all features) to 97.9% accuracy (top 3 features) — showing that removing weaker features actually improved performance.

Hence we can conclude that feature selection simplified the models without losing accuracy and even helped k-NN perform better.